In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import geemap
import ee
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import folium 
from datetime import date
import geojson

ModuleNotFoundError: No module named 'pandas'

In [ ]:
ee.Authenticate()
ee.Initialize()

: 

In [ ]:
gMap = geemap.Map()
gMap.add_basemap('HYBRID')

: 

In [ ]:
collection = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [[77.35549135039126,23.160079511460413],[77.35428972075259,23.15818557267024],[77.35583467314518,23.156133775428405],[77.35600633452212,23.1532927735212],[77.3494832021979,23.152503595633313],[77.34673662016665,23.150609549737823],[77.34433336088931,23.146347848590086],[77.3494832021979,23.149188997746933],[77.35360307524478,23.150451711371385],[77.35514802763736,23.148715477070244],[77.35626382658755,23.151114570670412],[77.35763711760318,23.149772943685907],[77.36004037688052,23.15214051165379],[77.36107034514224,23.154429120924355],[77.36278695891177,23.15703335288454],[77.36313028166568,23.15995318912088],[77.36536187956608,23.162872961707187],[77.37094087431704,23.16358316703569],[77.37205667326724,23.16658177023097],[77.37137002775943,23.167449774412837],[77.37102670500552,23.169738122100544],[77.36862344572818,23.172263150018228],[77.36793680022036,23.17044829101624],[77.36519021818911,23.171316270130436],[77.36347360441958,23.17242096268168],[77.36519021818911,23.174472510375498],[77.36338777373112,23.174472510375498],[77.36021203825747,23.174235794938678],[77.35849542448794,23.1763662187985],[77.3593537313727,23.17968014406207],[77.35926790068423,23.181337075918535],[77.35660714934146,23.180942570194695],[77.3560921652106,23.178259900426426],[77.35283059904849,23.177076352549115],[77.34974069426333,23.177313062961947],[77.3483674032477,23.17833880324607],[77.34527749846255,23.176287314815827],[77.34871072600161,23.176602930467386],[77.34871072600161,23.174551415428077],[77.35008401701724,23.175182654174062],[77.35231561491763,23.173446740459088],[77.35111398527896,23.170921734867097],[77.35343141386782,23.17139517704356],[77.35420389006411,23.173288929004993],[77.35626382658755,23.172736587450434],[77.35523385832583,23.170921734867097],[77.3586670858649,23.171079549111965],[77.35832376311099,23.1694224902621],[77.35858125517642,23.167607592750436],[77.35454721281802,23.165555939852318],[77.35394639799868,23.16310969723518],[77.34922571013247,23.1619260154104],[77.35549135039126,23.160079511460413]]
      }
    }
  ]
}

: 

In [ ]:
coords = collection['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

: 

In [ ]:
image = ee.Image(ee.ImageCollection("COPERNICUS/S2_SR") \
                      .filterBounds(aoi) \
                      .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',20)) \
                      .first())

ndwi = image.normalizedDifference(['B3', 'B8'])
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1= ndwiMasked.toInt()
vectors = ndwiMasked1.reduceToVectors(scale = 100.0, geometryType = 'polygon', eightConnected = False, maxPixels = 10000000, bestEffort=True)



: 

In [ ]:
start_date = '2000-01-01'
end_date = '2020-01-01' 



gMap.addLayer(aoi)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
               .filterDate(start_date,end_date) \
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
               .median()

ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')

ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')


ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')

gMap.addLayer(ndsi,{'min':-1,'max':1,'palette':['red','white','blue']},'ndsi')
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
                .filterDate(start_date,end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                .median()

: 

In [ ]:
PH = ee.Image.pixelLonLat().addBands(ph)

PH = PH.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);

data_ph = np.array((ee.Array(PH.get("ph")).getInfo()))

: 

In [ ]:
sal = ee.Image.pixelLonLat().addBands(ndsi)

sal = sal.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);

data_ndsi = np.array((ee.Array(sal.get("ndsi")).getInfo()))

: 

In [ ]:
df = pd.concat([pd.DataFrame(data_ndsi, columns = ['Salinity']),\
           pd.DataFrame(data_ph, columns = ['pH'])], axis=1, sort=False)


df.to_csv('Mahuakheda_lake_ph&salanity.csv',index=False)



: 

In [ ]:
visualization = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
  }
gMap.addLayer(sentinel, visualization, 'sent2rgb')
gMap.setCenter( 77.36076462415117, 23.16301066702798 , zoom = 12)
gMap


: 

: 